In [ ]:
import codecs
import sys
import urllib.request
import bs4
import os
import time
import re
from selenium import webdriver

In [ ]:
#URL
LIST_URL = 'https://psnprofiles.com/leaderboard/all?page='
MAIN_URL = 'https://psnprofiles.com'

In [ ]:
#ディレクトリ名
GAME_PATH = '../../psn_game'
USER_PATH = GAME_PATH + '/user' 
DATA_PATH = GAME_PATH + '/data'

In [ ]:
#重くなるから平均だけとる
game_trofy = []
#出現回数
game_count = []
#ユーザ名リスト
user_id_list = []
#ゲームURL
game_urls = []
#game_information=[['game_url',image_url',['ps4','ps3','vita']]]
game_information = []

In [ ]:
filename_games = DATA_PATH + '/games.txt'
filename_games2 = DATA_PATH + '/games+.txt'
filename_images = DATA_PATH + '/images.txt'
filename_platform = DATA_PATH + '/platform.txt'
filename_average = DATA_PATH + '/average.txt'
filename_count = DATA_PATH + '/count.txt'
filename_game_url = DATA_PATH + '/games_url.txt'
filename_users = DATA_PATH + '/users.txt'

In [ ]:
#重くなるから平均だけとる
game_trofy = []
#出現回数
game_count = []
#ユーザ名リスト
user_id_list = []
#ゲームURL
game_urls = []
#game_information=[['game_url',image_url',['ps4','ps3','vita']]]
game_information = []
#読み出し
f = open(filename_average,"r")
for x in f:
    game_trofy.append(x.rstrip("\n"))
f.close()

f = open(filename_count,"r")
for x in f:
    game_count.append(int(x.rstrip("\n")))
f.close()

f = open(filename_users,"r")
for x in f:
    user_id_list.append(x.rstrip("\n"))
f.close()

f = open(filename_game_url,'r')
for x in f:
    game_urls.append(x.rstrip("\n"))
    
f1 = open(filename_games2, 'r')
f2 = open(filename_images, 'r')
f3 = open(filename_platform, 'r')
for (i,j,k) in zip(f1,f2,f3):
    li = []
    li.append(i.strip('\n'))
    li.append(j.strip('\n'))
    l = []
    l.append(int(k[0]))
    l.append(int(k[2]))
    l.append(int(k[4]))
    li.append(l)
    game_information.append(li)

In [ ]:
#ページをクロール
def get_userlist(start, pages):
    #ユーザをリストに
    id_list = []
    for i in range(start, start+pages):
        #leaderboadから取得する
        list_url = LIST_URL + str(i)
        try:
            page_html = urllib.request.urlopen(list_url).read()
        except Exception as e:
                estr = str(i) + " is 404"
                print(estr)
                continue
        soup = bs4.BeautifulSoup(page_html,'lxml')
        table = soup.find('table', id ='leaderboard')
        user_list = table.find_all('a', class_ = 'title')
        for href in user_list:
            id = href.get('href')
            #既にとったIDならリストに加えない
            if user_id_list.count(id.strip('/'))==0:
                #ユーザをリストに加える
                id_list.append(id)
    return id_list

In [ ]:
#ページスクロール soupを返す
def get_page_scroll(url):
    #PhantomJs
    #https://gist.github.com/julionc/7476620
    driver = webdriver.PhantomJS()
    driver.get(url)
    #スクロールダウン
    #スクロールされてるか判断する部分
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    soup = []
    while True:
        #スクロールダウン
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  
        #読み込まれるのを待つ
        time.sleep(5)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        page_html = driver.page_source.encode('utf-8')
        soup = bs4.BeautifulSoup(page_html,'lxml')
        #スクロールされてるか判断する部分
        if newHeight == lastHeight:
            break
        lastHeight = newHeight
    return soup

In [ ]:
#平均トロフィー取得率取得
def get_average_trofy(arg):
    try:
        url = MAIN_URL+arg
        print(url)
        regex = r'[^\x00-\x7F]'
        matchedList = re.findall(regex,url)
        for m in matchedList:
            url = url.replace(m, urllib.parse.quote_plus(m, encoding="utf-8"))
        game_urls.append(url)
        soup=bs4.BeautifulSoup(urllib.request.urlopen(url).read(),'lxml')
    except Exception as e:
        estr=str(arg)+" is 404"
        print(estr)
        return
    stats = soup.find('div', attrs = {'class':'stats flex'}).get_text()
    tmp = re.findall(r'\d\d?%Average Completion',stats)
    str = tmp[0].replace('%Average Completion','')
    game_trofy.append(str)

In [ ]:
#game_information=[['game_url',image_url',['ps4','ps3','vita']]]
#game_information = []
#game_trofy=[owners,recent players, platinum achievers, average, earned]
#重くなるから平均だけとる
#game_trofy = []
#get_info('/mmilkkr', GAME_PATH + '/mmilkkr.txt')
#print(game_trofy)

In [ ]:
#ユーザの情報取得
def get_info(id, filename):
    #ユーザ名を1行めに書き込む
    f = open(filename, 'w')
    #ユーザ情報
    user_list = []
    #URL
    url = MAIN_URL + id 
    soup = get_page_scroll(url)
    #ゲーム画像とゲーム名
    #tableタグ取得
    Table = soup.find('table', attrs={'class':'zebra', 'id':'gamesTable'})
    if Table==None:
        print('private user')
        return -1
    user_id_list.append(id.strip('/'))
    #トロフィー平均取得率を得るためのゲームURL
    Game = Table.find_all('div', attrs={'class':'ellipsis'})
    #pictureタグ取得
    PIC = Table.find_all('picture', attrs={'class':'game'})
    #ユーザトロフィー取得率
    TOR = Table.find_all('div', attrs={'class':'progress-bar'})
    #プラットフォーム取得
    PLA = Table.find_all('div', attrs={'class':'platforms'})
    for (i,pic) in enumerate(PIC):
        LIST = []
        #imgタグ取得
        IMG = pic.find('img')
        #ゲームURL
        #game_name=Game[i].get_text().replace('\n','').replace('\t','').replace('•', '・')
        game_url = Game[i].a.get('href').replace(id,'')
        #print(MAIN_URL+'/'+game_url)
        #ゲームの追加
        LIST.append(game_url)
        #ゲーム画像URL追加
        try:
            LIST.append(IMG['src'])
        except:
            print(id,' image url none:')
            None
        #プラットフォーム取得
        try:
            Pla = [0,0,0]
            for pla in PLA[i].find_all('span'):
                if 'PS4' in pla:
                    Pla[0]=1
                elif 'PS3' in pla:
                    Pla[1]=1
                elif 'Vita' in pla:
                    Pla[2]=1
                else:
                    pass
            LIST.append(Pla)
        except:
            print(id,' platform none:')
            None
        if user_list.count(LIST)==0:
            user_list.append(LIST)
        #まだ取得していなゲームなら追加
        if game_information.count(LIST)==0:
            #平均トロフィー取得率取得
            get_average_trofy(game_url)
            game_information.append(LIST)
            index = game_information.index(LIST)
            game_count.append(1)
            print(index, game_count[index])
        else:
            index = game_information.index(LIST)
            game_count[index]=game_count[index]+1
            print(index, game_count[index])
    #ユーザトロフィー
    for (i,list) in enumerate(user_list):
        tor = TOR[i].find('span').get_text().strip('%')
        f.write(str(game_information.index(list))+'\t'+tor+'\n')

In [ ]:
if __name__ == '__main__':
    CURRENT = 2000
    try:
        os.makedirs(USER_PATH)
        os.makedirs(DATA_PATH)
    except FileExistsError:
        pass
    #ページ指定
    #start_pages=[100]
    start_pages=[1000,2000,3000,10000,15000,17000,20000,30000]
    #何人ずつ得るか
    #number_of_users=[2]
    number_of_users = [250,250,250,250,250,250,250,250]
    #合計人数
    all_number_of_user = 0
    for i in number_of_users:
         all_number_of_user = all_number_of_user + i
    j=0
    for (start, number) in zip(start_pages,number_of_users):
        #leaderboad,1ページ辺り50人得られる 第二引数10で500人
        id_list = get_userlist(start, 20)
        i=0
        for id in id_list:
            print(MAIN_URL+id)
            usercount = i+number*j+CURRENT
            fname = USER_PATH + '/' + str(usercount) + '.txt'
            #ユーザ情報が非公開ならインクリメントしない
            if get_info(id, fname) == -1:
                i=i-1
            i=i+1
            if i == number:
                break
        j = j+1
    print('ゲーム数:\t',len(game_information),'\tユーザ数:', all_number_of_user+CURRENT)

In [ ]:
fgame = open(filename_games, 'w')
fgame2 = open(filename_games2, 'w')
fimage = open(filename_images, 'w')
fplatform = open(filename_platform, 'w')
faverage = open(filename_average, 'w')
for i in range(len(game_information)):
    tmp = re.sub(r'/trophies/\d*-','',game_information[i][0])
    fgame.write(tmp+'\n')
    fgame2.write(game_information[i][0]+"\n")
    fimage.write(game_information[i][1]+'\n')
    for j in range(0,3):
        fplatform.write(str(game_information[i][2][j])+'\t')
    fplatform.write('\n')
for i in range(len(game_trofy)):
    faverage.write(game_trofy[i]+'\n')
fgame.close()
fgame2.close()
fimage.close()
fplatform.close()
faverage.close()
#print(game_information)

In [ ]:
fcount = open(filename_count, 'w')
fgame_url = open(filename_game_url, 'w')
fgame_users = open(filename_users, 'w')
for i in game_count:
    fcount.write(str(i)+'\n')
fcount.close()
for i in game_urls:
    fgame_url.write(str(i)+'\n')
fgame_url.close()
for i in user_id_list:
    fgame_users.write(str(i)+'\n')
fgame_users.close()

In [ ]:
import pandas as pd
filename_all = GAME_PATH + '/all.csv'
df = pd.DataFrame(game_information)
df.to_csv(filename_all, index=False)

In [ ]:
df = pd.read_csv(filename_all)
list1 = df.values.tolist()

In [ ]:
get_info('/mmilkkr', DATA_PATH + '/mmilkkr.txt')